<a href="https://colab.research.google.com/github/arturLoza/MCMC-methods-in-option-valuation/blob/main/v2_M5U1_Tarea_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src = "https://drive.google.com/uc?export=view&id=1QqjbbEZ1w7xoawV020Jj_R46PKRi6A_e" alt = "Encabezado MLDS" width = "100%">  </img>

# **Tarea 1: Clasificación lineal**
---

En esta tarea deberá entrenar modelos de clasificación con regresión logística para el [conjunto de datos de vinos Wine](https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data) del repositorio de la *UCI* usando *Keras* con el back end de *TensorFlow*.

Ejecute las siguientes celdas para conectarse a UNCode:

In [1]:
%%capture
!pip install rlxcrypt
!wget --no-cache -O session.pye -q https://raw.githubusercontent.com/JuezUN/INGInious/master/external%20libs/session.pye

In [2]:
import rlxcrypt
import session

grader = session.LoginSequence('DL-GroupMLDS-5-2025-2@c1f23e8b-1364-4562-bf6f-2bc3bf4d065b')

Please enter your UNCode username: bacamargol
Please enter your password: ··········


Ejecute la siguiente celda para importar y configurar las librerías usadas :

In [3]:
# Librerías de utilidad para manipulación y visualización de datos.
from numbers import Number
import numpy as np
import tensorflow as tf
import keras
import matplotlib as mpl
import matplotlib.pyplot as plt

from numpy.random import seed
seed(1)

# Ignorar warnings.
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Versiones de las librerías usadas.
!python --version
print('Tensorflow', tf.__version__)
print('Keras', keras.__version__)

Python 3.12.11
Tensorflow 2.19.0
Keras 3.10.0


Esta actividad se realizó con las siguientes versiones:
*  Python 3.11.12
*  Tensorflow 2.18.0
*  Keras 3.8.0

## **Cargar los datos**
---
En el conjunto *Wine* las características de entrada corresponden a diferentes atributos del vino. El conjunto de datos contiene 178 ejemplos sobre los que se tiene la siguiente información :

* Alcohol
* Malic acid
* Ash
* Alcalinity of ash
* Magnesium
* Total phenols
* Flavanoids
* Nonflavanoid phenols
* Proanthocyanins
* Color intensity
* Hue
* OD280/OD315 of diluted wines
* Proline

Hay tres clases de vinos diferentes. `class_0` el cual tiene 59 muestras, `class_1` el cual tiene 71 muestras y `class_2` el cual tiene 48 muestras.

Como en cualquier experimento de _machine learning_, vamos a empezar cargando el conjunto de datos, haciendo particiones de entrenamiento y prueba, y para efectos de esta tarea, nos vamos a quedar solo con dos clases (`class_1` y `class_2`) para hacer clasificación:

In [5]:
!wget --no-cache -O wine.data -q  https://raw.githubusercontent.com/mindlab-unal/mlds5-datasets/main/u1/taller/wine.data?raw=true

In [6]:
import pandas as pd
from sklearn import preprocessing
from sklearn import model_selection

# Leer el archivo que contiene los datos:
data =  pd.read_csv('wine.data', sep=",", header=None)
# La etiqueta está consignada en la primera columna:
X_all = np.array(data.iloc[:,1:])
y_all = np.array(data.iloc[:,0])
# Nos quedamos con la clase 2 y 3, y ajustamos las etiquetas para que queden
# como 0 y 1:
X = X_all[np.where((y_all==2)|(y_all==3))]
y = y_all[np.where((y_all==2)|(y_all==3))]-2
# Re-escalamos los datos
scaler = preprocessing.MinMaxScaler((0, 1))
X = scaler.fit_transform(X)
# Y hacemos partición en entrenamiento y prueba
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

Verifiquemos el tamaño de las particiones:

In [7]:
print("Número de muestras de entrenamiento =", X_train.shape[0])
print("Número de muestras de prueba =", X_test.shape[0])
print("Número de características del conjunto de datos =", X_train.shape[1])

Número de muestras de entrenamiento = 83
Número de muestras de prueba = 36
Número de características del conjunto de datos = 13


**Salida esperada**

```python
Número de muestras de entrenamiento = 83
Número de muestras de prueba = 36
Número de características del conjunto de datos = 13
```

Tenemos entonces 122 muestras con 13 _features_ para trabajar. Ahora veamos cuántas muestras hay por cada clase:

In [8]:
print("Número de muestras de la clase 2 en entrenamiento =", X_train[np.where(y_train==0)].shape[0])
print("Número de muestras de la clase 3 en entrenamiento =", X_train[np.where(y_train==1)].shape[0])
print("Número de muestras de la clase 2 en prueba =", X_test[np.where(y_test==0)].shape[0])
print("Número de muestras de la clase 3 en prueba =", X_test[np.where(y_test==1)].shape[0])

Número de muestras de la clase 2 en entrenamiento = 50
Número de muestras de la clase 3 en entrenamiento = 33
Número de muestras de la clase 2 en prueba = 21
Número de muestras de la clase 3 en prueba = 15


**Salida esperada**
```python
Número de muestras de la clase 2 en entrenamiento = 50
Número de muestras de la clase 3 en entrenamiento = 33
Número de muestras de la clase 2 en prueba = 21
Número de muestras de la clase 3 en prueba = 15
```

Como puede ver, el conjunto de datos está desbalanceado. Vamos entonces a implementar un modelo que compense este desbalance desde la función de pérdida.

## **Modelo de regresión logística con _class_weight_**
---
Una vez se dispone de un conjunto de datos preparado para el entrenamiento, se declara el algoritmos de aprendizaje computacional. En nuestro caso queremos predecir el valor de una variable categórica, es decir, realizar un modelo para **clasificación**.

Sin embargo, el conjunto de datos no está balanceado. Cuando esto sucede, podemos compensar el desbalance dándole más importancia a la clase menos presente. Darle más importancia a una clase se logra asignando un peso por cada clase dentro de la función de pérdida del modelo.

Supongamo que tenemos un problema desbalanceado de clasificación binario con etiquetas $0$ y $1$. Supongamos que $n_0$ es el número de elementos de la clase $0$ y $n_1$ es el número de elementos de la clase $1$. Una elección convencional sobre los pesos que se le deben asignar a cada clase es :

$$w_0=\dfrac{n_0+n_1}{2n_0},$$

$$w_1=\dfrac{n_0+n_1}{2n_1}.$$

Y estos pesos se incorporan a la función de pérdida de la siguiente forma :

$$\mathcal{L}(\vec{w})=-\frac{1}{N}\sum_{i=1}^{N}[w_1 y_i\log(\tilde{y}_i)+w_0(1-y_i)\log(1-\tilde{y}_i)],$$

¿Cómo funciona? Supongamos que hay 100 muestras de la clase $0$ y 50 de la clase $1$. Es decir, $n_0=100$ y $n_1=50$. Entonces $w_0=0.75$ y $w_1=1.5$. Así, el peso de la clase $1$ es el doble del peso de la clase $0$. Cuando el modelo no clasifica bien una muestra de la clase $1$, la penalidad se multiplica por $w_1$. Es decir, el modelo entiende que es más grave equivocarse con los datos de la clase $1$, y de esa manera compensa el que sean menos muestras que las de la clase $0$, tratanto de previnir cualquier tipo de sesgo en el modelo final.



> **La tarea es incremental, por lo tanto es recomendable resolver los puntos en orden**

## **1. Calcular el peso de cada clase**
---

Complete la función **`class_weights`** para que calcule los pesos que el modelo tiene que darle a cada clase según el desbalance de los datos.

**Entrada** :

* **`y`**: un `numpy.ndarray` de tamaño `(m,)`; es decir, el vector de etiquetas de los datos de entrenamiento. $m$ el número de muestras del conjunto de entrenamiento.

**Salida** :

* **`weights_list`** : `list`, una lista con los pesos (tipo `float`) de cada clase, en orden.

0.5845070422535211

In [37]:
# FUNCIÓN CALIFICADA class_weights:

def class_weights(y):
    # Reemplazar con respuesta
    w_0 = (y[y==0].shape[0] + y[y==1].shape[0]) / (2*y[y==0].shape[0])
    w_1 = (y[y==0].shape[0] + y[y==1].shape[0]) / (2*y[y==1].shape[0])
    weights_list = [w_0, w_1]
    return weights_list

In [38]:
#TEST_CELL

class_weights_list = class_weights(y_train)
print(np.round(class_weights_list, 5))

[0.83    1.25758]


**Salida esperada**:

```python
[0.83    1.25758]
```

### **Evaluar código**

In [39]:
grader.run_test("Test 1_1", globals())

Test 1_1


In [40]:
grader.run_test("Test 1_2", globals())

Test 1_2


## **2. Binary cross-entropy**
---
Ahora, complete la función **`get_loss`**. Esta debera dar como resultado una función que retorne el valor de la entropia cruzada, teniendo en cuenta los pesos de cada clase.

**Entrada**:

* **`class_weights`** : `list`, una lista con los pesos asociados a cada clase.

**Salida**:

* **`loss`** : `function`, una función que calcula la entropia cruzada ponderada y recibe unicamente los paramteros **`y_true`** y **`y_pred`**

$$w_0=\dfrac{n_0+n_1}{2n_0},$$

$$w_1=\dfrac{n_0+n_1}{2n_1}.$$

Y estos pesos se incorporan a la función de pérdida de la siguiente forma :

$$\mathcal{L}(\vec{w})=-\frac{1}{N}\sum_{i=1}^{N}[w_1 y_i\log(\tilde{y}_i)+w_0(1-y_i)\log(1-\tilde{y}_i)],$$

In [95]:
# FUNCIÓN CALIFICADA weighted_bce:
import keras
import keras.ops as K
import numpy as np
def get_loss(class_weights):
    import keras.ops as K
    w0, w1 = class_weights

    def loss(y_true, y_pred):
        weighted_bce  = K.mean( -w1 * y_true * np.log(y_pred) - w0 * (1 - y_true) * np.log(1 - y_pred) )
    # Reemplazar con respuesta
        return  weighted_bce
    return loss

Use las siguientes celdas para probar su modelo:

In [96]:
#TEST_CELL
class_weights_example = class_weights(y_train)
Y_t = np.expand_dims(y_train, axis=-1)
y_true = Y_t
y_pred = np.ones([Y_t.shape[0],1]) * 0.5
loss = get_loss(class_weights_example)
print("weighted_bce =", loss(y_true,y_pred).numpy())

weighted_bce = 0.6931471805599454


In [114]:
import keras

def get_loss(class_weights):
    w0, w1 = class_weights

    def loss(y_true, y_pred):
        # Usar keras.ops en lugar de numpy
        weighted_bce = keras.ops.mean(
            -w1 * y_true * keras.ops.log(y_pred)
            - w0 * (1 - y_true) * keras.ops.log(1 - y_pred)
        )
        return weighted_bce

    return loss


**Salida esperada**:

```python
weighted_bce = 0.6931471

```

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>


* Utilice la función [**`keras.ops.mean`**] de la guia de _keras_.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>


* El uso de una función de pérdida de entropía cruzada ponderada implica el uso de los pesos de cada clase para hallar su valor.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 3</b></font>
</summary>


* Tenga en cuenta que la salida de esta función debe ser otra función


### **Evaluar código**

In [115]:
grader.run_test("Test 2_1", globals())

Test 2_1


In [98]:
grader.run_test("Test 2_2", globals())

Test 2_2


## **3. Optimizador**
---
Complete la función **`optimizer`** para que devuelva un optimizador válido. Deberá específicar el tipo de optimizador y la tasa de aprendizaje que usaremos en el entrenamiento.

**Entrada** :

* **`type_opt`** : `str`, que puede tomar valores entre: **`SGD`**, **`Adam`**, **`RMSprop`**.
* **`learning_rate`** : `float`, correspondiente a la tasa de aprendizaje.

**Salida** :

* **`opt`** : El optimizador con tasa de aprendizaje definida, un objeto tipo `keras.optimizers`.

In [101]:
import keras
def optimizer(type_opt, learning_rate):
    # Obtener la clase del optimizador desde keras.optimizers usando su nombre
    opt_class = getattr(keras.optimizers, type_opt)
    # Instanciar con la tasa de aprendizaje
    opt = opt_class(learning_rate=learning_rate)
    return opt

Use las siguientes celdas para probar su modelo:

In [102]:
#TEST_CELL

type_opt_test = 'Adam'
learning_rate_test = 0.5
optimizer(type_opt_test,learning_rate_test)

**Salida esperada**:

```
<keras.optimizers.adam.Adam at 0x7f53eddd39d0>
```

### **Evaluar código**

In [103]:
grader.run_test("Test 3_1", globals())

Test 3_1


In [104]:
grader.run_test("Test 3_2", globals())

Test 3_2


## **4. Defina el modelo**
---
Teniendo una función de coste y un optimizador, podemos generar un modelo. A continuación implementa la función **`get_model`**, en la cual deberá definir el modelo usando _Keras_, considere que este se trata de una regresión logística.

**Entrada** :

* **`n_inputs`** : `int`, el número de vairables que utilizrá el modelo como entrada.
* **`n_neurons`** : `int`, el número de salidas esperadas.
* **`optimizer`** : `keras.optimizers`, el optimizador definido que se usará para minimizar la función de pérdida **`loss_fun`**
* **`loss`**: `function`, Función de perdida a optimizar.

**Salida** :

* **`model`** : `keras.Models.Sequential` Modelo a entrenar.

In [105]:
def get_model(n_inputs, n_neurons, optimizer, loss):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(n_neurons, activation='sigmoid', input_shape=(n_inputs,)))
    model.compile(loss=loss, optimizer=optimizer)
    return model

In [106]:
#TEST_CELL
class_weights_example = class_weights(y_train)
loss = get_loss(class_weights_example)
opt = optimizer('Adam',0.5)
model = get_model(13, 1, opt, loss)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1)              │            14 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14 (56.00 B)

 Trainable params: 14 (56.00 B)

 Non-trainable params: 0 (0.00 B)

**Salida esperada**

```
Model: "sequential"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 1)                   │              14 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘
 Total params: 14 (56.00 B)
 Trainable params: 14 (56.00 B)
 Non-trainable params: 0 (0.00 B)
```

### **Evaluar código**

In [107]:
grader.run_test("Test 4_1", globals())

Test 4_1


In [108]:
grader.run_test("Test 4_2", globals())

Test 4_2


## **5. Entrenamiento del modelo**
---
Una vez definida cada una de las piezas, podemos entrenar el modelo. Complete la función **`train_model`** para que retorne los pesos, del modelo entrenado sobre los arreglos **`X_t`** y **`Y_t`**.

**Entrada** :

* **`model`** : `keras.Models.Sequential` Modelo a entrenar.
* **`epochs`** : `int`, el número de iteraciones durante las cuales se realizará el entrenamiento del modelo.
* **`optimizer`** : `keras.optimizers`, el optimizador definido que se usará para minimizar la función de pérdida **`loss_fun`**
* **`X_t`**: `Tensor`, tensor de tamaño `(m,n)`, correspondiente a la matriz de datos de entrenamiento.
* **`Y_t`**: `Tensor`, tensor de tamaño `(m,1)`, correspondiente a las etiquetas de los datos entrenamiento.
* **`loss`** : `function`, una función que calcula la entropia cruzada ponderada y recibe unicamente los paramteros **`y_true`** y **`y_pred`**




**Salida** :

* **`losses`** : `list` Una lista con los valores de **`loss_fun`** en cada iteración.
* **`weigths`**: `list`, lsita de tensores que contiene los pesos entrandos del mdoelo.
 >nota: Utiliza como referencia la construcción del modelo basado en Keras del taller guiado.


In [116]:
import numpy as np

def train_model(model, epochs, optimizer, X_t, Y_t, loss):
    # Entrenar el modelo
    history = model.fit(
        x=X_t.astype(np.float32),   # Datos de entrada
        y=Y_t.astype(np.float32),   # Etiquetas de salida
        epochs=epochs,
        verbose=0
    )

    # Obtener las pérdidas (losses) registradas por época
    losses = history.history['loss']

    # Obtener los pesos entrenados del modelo
    weights = model.get_weights()

    return losses, weights


In [117]:
#TEST_CELL
keras.utils.set_random_seed(1) ## Fijar semilla para obtener siempre el mismo valor
class_weights_example = class_weights(y_train)
loss = get_loss(class_weights_example)
opt = optimizer('Adam',0.5)

model = get_model(13, 1, opt, loss)
model.layers[0].set_weights([np.zeros([13,1]),np.array([0])])

X_t = X_train
Y_t = np.expand_dims(y_train, axis=-1)
losses, weigths = train_model(model, 10, opt, X_t, Y_t, loss)
print('Epoch', 0, 'loss =', losses[0])
print('Epoch', 1, 'loss =', losses[1])
print("w[0] =", weigths[0][0])
print("b =",weigths[1])

Epoch 0 loss = 0.9399663209915161
Epoch 1 loss = 0.3858067989349365
w[0] = [2.2023726]
b = [-0.09913202]


**Salida esperada**:

```
Epoch 0 loss = 0.9399663209915161
Epoch 1 loss = 0.3858067989349365
w[0] = [2.2023726]
b = [-0.09913202]
```

### **Evaluar código**

In [118]:
grader.run_test("Test 5_1", globals())

Test 5_1


In [119]:
grader.run_test("Test 5_2", globals())

Test 5_2


## **6. Predicciones del modelo**
---
Finalmente, podemos utilizar los datos reservados de la partición de prueba para calcular predicciones y hacer evaluaciones.

Complete la función **`model_predict`** para que retorne las predicciones del modelo entrenado sobre un conjunto de prueba.

**Entrada** :
* **`model`** : `keras.Models.Sequential` Modelo a entrenar.
* **`X_test`** : `Tensor`, tensor de tamaño `(l,n)`, correspondiente a la matriz de datos de prueba.

**Salida** :

* **`y_pred`** : Tensor de tamaño `(l,1)`, con las predicciones de **model** para el conjunto de prueba **`X_test`**.

In [125]:
import tensorflow as tf

def model_predict(model, X_test):
    # Asegurarnos de que X_test es un tf.Tensor con dtype float32
    X_t = tf.convert_to_tensor(X_test, dtype=tf.float32)
    # Llamar al modelo directamente devuelve un tf.Tensor (no numpy)
    y_pred = model(X_t, training=False)
    return y_pred


Use la siguiente celda para probar su función:

In [126]:
keras.utils.set_random_seed(1)
class_weights_example = class_weights(y_train)
loss = get_loss(class_weights_example)
opt = optimizer('Adam',0.5)
model = get_model(13, 1, opt, loss)
model.layers[0].set_weights([np.zeros([13,1]),np.array([0])])

losses, weigths = train_model(model, 10, opt, X_t, Y_t, loss)
y_pred = model_predict(model, X_t)

print("Primeras dos predicciones:\n", y_pred[:2].numpy())
m = keras.metrics.Accuracy()
m.update_state(Y_t, tf.math.round(y_pred))
print("Accuracy sobre X_test después de 10 epochs:", m.result().numpy())

Primeras dos predicciones:
 [[0.9925333]
 [0.0018804]]
Accuracy sobre X_test después de 10 epochs: 0.9879518


**Salida esperada:**

```
Primeras dos predicciones:
 [[0.9925333]
 [0.0018804]]
Accuracy sobre X_test después de 10 epochs: 0.9879518
```

### **Evaluar código**

In [127]:
grader.run_test("Test 6_1", globals())

Test 6_1


In [128]:
grader.run_test("Test 6_2", globals())

Test 6_2


# **Evaluación**

In [129]:
grader.submit_task(globals())

Test 1_1
Test 1_2
Test 2_1
Test 2_2
Test 3_1
Test 3_2
Test 4_1
Test 4_2


Test 5_1
Test 5_2
Test 6_1
Test 6_2


# **Créditos**
---

* **Profesor:** [Fabio Augusto Gonzalez](https://dis.unal.edu.co/~fgonza/)
* **Asistentes docentes :**
  * [Juan Sebastián Malagón Torres](https://sites.google.com/unal.edu.co/santiagotoledo-cortes/)
  * [Santiago Toledo Cortés](https://sites.google.com/unal.edu.co/santiagotoledo-cortes/)
* **Diseño de imágenes:**
    - [Mario Andres Rodriguez Triana](mailto:mrodrigueztr@unal.edu.co).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*